In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename='/data/mine/vadeec/merged/ukdale.h5',
        buildings=[1, 2],
        sample_period=6,
        window=("2013-04-01", "2013-05-01")
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 164 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 174 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 24 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 94 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 0 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:

In [3]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance='kettle',
    seq_length=2048
)

In [4]:
seq = source.get_sequence(enable_all_appliances=True)

In [5]:
all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

True

In [6]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#plt.show()

In [7]:
sample = source.get_batch(num_seq_per_batch=1024)

In [8]:
pipeline = DataPipeline(
    source, 
    num_seq_per_batch=64,
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

In [9]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'input_processing': [],
  'num_seq_per_batch': 64,
  'source': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
    'allow_incomplete_target': True,
    'distractor_inclusion_prob': 1.0,
    'include_incomplete_target_in_output': True,
    'issued_validation_warning': False,
    'rng_seed': None,
    'seq_length': 2048,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 1.0,
    'uniform_prob_of_selecting_each_model': True}},
  'target_processing': [{'DivideBy': {'divisor': 295.34958}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'buildings': [1, 2],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'window': ('2013-04-01', '2013-05-01')}}}

In [10]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {'dense_layer_0': 100}
    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [11]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [12]:
from neuralnilm.trainer import Trainer

PATH = '/home/dk3810/temp/neuralnilm'

trainer = Trainer(
    net,
    data_pipeline=pipeline,
    path=PATH,
    learning_rate=1E-4,
    repeat_callbacks=[
        (  10, Trainer.validate),
        (  10, Trainer.plot_costs)
    ]
)

In [13]:
trainer.fit(50)

INFO:neuralnilm.trainer:Starting training for 50 iterations.



 Update |  Train cost  |  Valid cost  |  Train / Val  | Secs per update
--------|--------------|--------------|---------------|----------------


INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.
INFO:neuralnilm.trainer:Compiling validation cost function...
INFO:neuralnilm.trainer:Done compiling cost function.


      0 |  33248.926537  |    7.659136  |  4341.080591  |  0.718s
      1 |    9.486720  |    7.659136  |     1.238615  |  0.023s
      2 |    4.498129  |    7.659136  |     0.587289  |  0.018s
      3 |    1.054216  |    7.659136  |     0.137642  |  0.022s
      4 |    2.642872  |    7.659136  |     0.345061  |  0.021s
      5 |    2.678279  |    7.659136  |     0.349684  |  0.051s
      6 |    2.575255  |    7.659136  |     0.336233  |  0.036s
      7 |    1.005692  |    7.659136  |     0.131306  |  0.051s
      8 |    0.939625  |    7.659136  |     0.122680  |  0.036s
      9 |    3.076981  |    7.659136  |     0.401740  |  0.034s
     10 |    0.909405  |    0.999975  |     0.909428  |  0.059s
     11 |    0.956710  |    0.999975  |     0.956734  |  0.036s
     12 |    1.560736  |    0.999975  |     1.560774  |  0.051s
     13 |    0.953144  |    0.999975  |     0.953167  |  0.040s
     14 |    1.079718  |    0.999975  |     1.079744  |  0.082s
     15 |    1.038911  |    0.999975  

INFO:neuralnilm.trainer:Stopped training. Completed 50 iterations.



     50 |    0.971561  |    0.999975  |     0.971585  |  0.038s


In [14]:
trainer.iteration

50

In [15]:
trainer.training_costs

[33248.926537290201,
 9.4867198892413942,
 4.4981287850773626,
 1.0542160456991987,
 2.6428719354835475,
 2.6782789502549864,
 2.5752552189153777,
 1.0056917521542497,
 0.93962538294621534,
 3.0769811650714254,
 0.90940534678908469,
 0.95671046534339388,
 1.5607359261325076,
 0.95314383303919814,
 1.0797177483835885,
 1.0389109484307124,
 1.0373694858195206,
 1.080630667129034,
 1.0329496980200941,
 0.9924322637046864,
 2.1950460222184489,
 4.1658200523824469,
 1.0510488196001624,
 0.97463688052599329,
 1.0949937076676246,
 1.5505119389536379,
 1.1681578165082869,
 1.0840633940242084,
 0.90874027647281541,
 0.9882184769723219,
 1.0115398345723947,
 0.97891889487903883,
 0.89928416276920686,
 0.96720935062505742,
 0.97635874753531593,
 1.7445087519033018,
 1.0253234832564708,
 1.0144209415573275,
 0.96332905804346514,
 0.85269410477277152,
 0.98219476726054433,
 1.0189890553769647,
 0.8836960084816069,
 0.93793231254207043,
 0.90229167221944606,
 1.1939027010107779,
 0.88369962854369843